# TOPIC MODELING

Las técnicas de clustering suelen utilizarse conjuntamente con lo que se denomina topic modeling.
El topic modeling; ayuda a conocer de forma automática cuáles son los temas principales de los que se está hablando en distintos textos.

In [2]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [3]:
#***************************************************************************************
# 1. Cargamos el corpus de textos
#***************************************************************************************

print("\n\n1. Cargamos el corpus")
trainCorpus = ["Me gustan las vacas",
               "Me gustan los caballos",
               "odio los perros",
               "odio los caballos",
               "me gustan las ranas",
               "me gusta el helado",
               "no quiero comer",
               "los helados son cremosos"]

print (trainCorpus)



1. Cargamos el corpus
['Me gustan las vacas', 'Me gustan los caballos', 'odio los perros', 'odio los caballos', 'me gustan las ranas', 'me gusta el helado', 'no quiero comer', 'los helados son cremosos']


In [4]:
#*********************************************************************************************************
# 2. Vectorizamos los textos del corpus (convertimos cada texto en un vector de frecuencias de palabras)
#*********************************************************************************************************
print("\n\n2. Vectorizamos los textos")

puncts = [c for c in string.punctuation]
spanish_stopwords = nltk.corpus.stopwords.words('spanish')+puncts
vectorizer = CountVectorizer(stop_words=spanish_stopwords)

#Transformamos los documentos en una matriz de tf's de documentos.
vectorizer.fit(trainCorpus) #El vectorizador aprende el vocabulario del corpus
print ("\nAtributos:",vectorizer.get_feature_names_out())
tfMatrix = vectorizer.transform(trainCorpus) #Extraemos las frecuencias de palabras (tf)
print ("\nMatriz tf:\n",tfMatrix.toarray())

#La matriz tf es nuestro dataset, donde:
# - cada fila representa una muestra (un documento del corpus)
# - cada columna representa un atributo (la frecuencia de una palabra en dicho documento)
print("\nnº de muestras: %d, nº de atributos: %d" % tfMatrix.shape)
tf_feature_names = vectorizer.get_feature_names_out()





2. Vectorizamos los textos

Atributos: ['caballos' 'comer' 'cremosos' 'gusta' 'gustan' 'helado' 'helados' 'odio'
 'perros' 'quiero' 'ranas' 'vacas']

Matriz tf:
 [[0 0 0 0 1 0 0 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0]]

nº de muestras: 8, nº de atributos: 12


In [5]:
#*********************************************************************************************************
# 3. Aplicamos el algoritmo LDA para extraer los topics
#*********************************************************************************************************
topics = 2
lda_model = LatentDirichletAllocation(n_components =topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tfMatrix)
H = lda_model.components_
W = lda_model.transform(tfMatrix)

no_top_words = 2
no_top_documents = 3

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\nTopic %d:" % (topic_idx))
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            print(" ",feature_names[i])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(trainCorpus[doc_index])

display_topics(H, W, tf_feature_names, trainCorpus, no_top_words, no_top_documents)


Topic 0:
  caballos
  odio
odio los caballos
odio los perros
Me gustan los caballos

Topic 1:
  gustan
  helado
no quiero comer
me gusta el helado
Me gustan las vacas


La salida de la ejecución muestra la matriz TF generada y los topics de cada documento.
Es decir, que, de alguna forma, hay documentos que hablan de “odiar caballos” y otros
hablan de “gustar helado”.